# Automatic Syllabus Classification

In [ ]:
from os import environ
from google.generativeai import configure, GenerativeModel
from requests import get

from syllabus_classificaiton.agents import agents
from syllabus_classificaiton.prompt import prompt

In [ ]:
competences = get("https://raw.githubusercontent.com/mckoh/digicomp2-2/refs/heads/main/competences.json").content

In [ ]:
lv_description = {
    "name": "MACHINE LEARNING & DEEP LEARNING",
    "objectives": "- Die Studierenden kennen Werkzeuge (z.B. Bibliotheken, Cloud Plattformen oder Softwarewerkzeuge), mit deren Hilfe Machine Learning unterstützt werden kann. - Die Studierenden können die erarbeiteten Werkzeuge hinsichtlich ihrer Eignung für konkrete Problemstellungen vergleichen. - Die Studierenden können Ende-zu-Ende Machine Learning Projekte konzipieren. - Die Studierenden können Ende-zu-Ende Machine Learning Projekte selbstständig durchführen."
}

In [ ]:
prompt = prompt.replace("**name**", lv_description["name"])
prompt = prompt.replace("**objectives**", lv_description["objectives"])

## Model Setup

In [ ]:
configure(api_key=environ["GEN_AI_KEY"])

model = GenerativeModel("models/gemini-2.5-flash")

## Collect Responses from Agents

In [ ]:
responses = []

for agent in agents:
    chat = model.start_chat(history=[])

    _ = chat.send_message(f"Hier stelle ich dir eine formale Beschreibung des DigiComp 2.2 Kompetenzrasters zur Verfügung: {competences}")

    prompt = prompt.replace("**role**", agent["name"])
    prompt = prompt.replace("**description**", agent["description"])

    response = chat.send_message(content=prompt, stream=False)
    responses.append(response.text)

In [ ]:
for i, response in enumerate(responses):
    with open(f"ergebnisse/agent_{i}.md", "w") as text_file:
        text_file.write(response)